In [ ]:
# 必要なライブラリのインポート
import tensorflow as tf
# GPU設定（CPU環境でも安全に動作）
print(f"TensorFlow version: {tf.__version__}")
try:
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        for gpu in gpus:
            try:
                tf.config.experimental.set_memory_growth(gpu, True)
            except RuntimeError as e:
                print(f"Memory growth setup failed for {gpu.name}: {e}")
        strategy = tf.distribute.OneDeviceStrategy(device="/GPU:0")
        gpu_names = [gpu.name for gpu in gpus]
        print(f"✓ GPU detected: {gpu_names}")
        print(f"✓ Using GPU acceleration")
    else:
        strategy = tf.distribute.get_strategy()
        print("✓ No GPU detected - running on CPU")
        print("  (For GPU support, use devcontainer-gpu.json)")
except Exception as e:
    # CPU環境での安全なフォールバック
    strategy = tf.distribute.get_strategy()
    print(f"✓ Running on CPU (GPU setup skipped: {e})")

In [ ]:
%load_ext autoreload
%autoreload 2
# カスタムモジュールのインポート
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import sys
from pathlib import Path

rootPath = Path.cwd().parent
sys.path.append(str(rootPath))

from src.timeseries_processing import SequenceCreator, DataSplitter, DataStandardizer, feature_groups_route_based
from src.model_training import DelayPredictionModel
from src.evaluation import ModelEvaluator

pd.set_option('display.max_columns', None)
delay_features = pd.read_csv(f'{rootPath}/data/merged_dataset.csv')

In [ ]:
# 時系列シーケンス作成（カスタムモジュール使用）
INPUT_TIMESTEPS = 8
OUTPUT_TIMESTEPS = 3
sequence_creator = SequenceCreator(
    input_timesteps=INPUT_TIMESTEPS, 
    output_timesteps=OUTPUT_TIMESTEPS,
    feature_groups=feature_groups_route_based  # カスタムfeature_groupsを指定
)

# 空間的配置を有効にしてシーケンス作成（feature_colsとtarget_colは自動生成）
X_delay, y_delay, route_direction_info, used_features, feature_group_info = sequence_creator.create_route_direction_aware_sequences(
    delay_features,
    spatial_organization=True  # ConvLSTM用の空間配置を有効化
)

print(f"使用された特徴量: {used_features}")
print(f"シーケンス形状: X={X_delay.shape}, y={y_delay.shape}")

# 特徴量グループの表示
if feature_group_info:
    print(f"\n=== ConvLSTM Feature Groups (Width Dimension) ===")
    for group_name, info in feature_group_info.items():
        print(f"{group_name.capitalize()}: {info['features']} -> width indices [{info['start_idx']}:{info['end_idx']}]")
        print(f"  Size: {info['size']} features")

In [ ]:
# データ分割
splitter = DataSplitter()

# すべてのRoute-direction内で分割
X_train, X_test, y_train, y_test, train_routes, test_routes = splitter.train_test_split_by_route_direction_stratified(
    X_delay, y_delay, route_direction_info, train_ratio=0.9
)

# 分割結果の検証
validation_results = splitter.validate_split(
    X_train, X_test, y_train, y_test, 
    route_direction_info, train_routes, test_routes
)

# サマリー表示
splitter.print_split_summary(validation_results)

In [ ]:
# データ標準化（特徴量のみ）
standardizer = DataStandardizer()
X_train_scaled = standardizer.fit_transform_features(X_train)
X_test_scaled = standardizer.transform_features(X_test)

In [ ]:
# ConvLSTM用reshape
actual_feature_count = X_train_scaled.shape[2]
X_train_reshaped = splitter.reshape_for_convlstm(
    X_train_scaled, target_height=1, target_width=actual_feature_count
) 
X_test_reshaped = splitter.reshape_for_convlstm(
    X_test_scaled, target_height=1, target_width=actual_feature_count
)

print(f"\n=== ConvLSTM Reshape Results ===")
print(f"Training data: {X_train_reshaped.shape} (samples, timesteps, height, width, channels)")
print(f"Test data: {X_test_reshaped.shape}")

In [ ]:
# 遅延予測用モデルの作成
model_trainer = DelayPredictionModel(
    input_timesteps=INPUT_TIMESTEPS,
    output_timesteps=OUTPUT_TIMESTEPS
)

# モデル構築
delay_model = model_trainer.build_model(X_train_reshaped.shape[1:])

In [ ]:
# モデル訓練
print("=== Delay Prediction Model Training Started ===")

history = model_trainer.train_model(
    X_train_reshaped, 
    y_train,
    batch_size=32, 
    epochs=50, 
    validation_split=0.2,
    model_path='best_delay_model.h5'
)

if history is not None:
    print("Training completed successfully!")
else:
    print("Training failed. Please check the error messages above.")

In [ ]:
# モデル評価・可視化
print("=== Delay Prediction Model Evaluation ===")

# 予測実行
predictions = model_trainer.predict(X_test_reshaped, batch_size=32)


In [ ]:
# 各時間ステップ（1時間後、2時間後、3時間後）での評価
evaluator = ModelEvaluator()
timestep_results = []
for timestep in range(OUTPUT_TIMESTEPS):
    y_pred_t = predictions[:, timestep]
    y_test_t = y_test[:, timestep]
    
    metrics = evaluator.calculate_delay_metrics(y_test_t, y_pred_t)
    delay_analysis = evaluator.analyze_by_delay_level(y_test_t, y_pred_t)
    
    timestep_results.append(evaluator.evaluation_summary(timestep, metrics, delay_analysis))

print(timestep_results[0])